In [ ]:
import time, json, glob, os
from py2neo import Graph, Node, Relationship
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set(color_codes=True)
%matplotlib inline

In [ ]:
graph = Graph("bolt://neo4j-allquanta:7687", auth=('neo4j','myneo'))

print("Connected to graph database with {:,} nodes and {:,} relationships!".format(
    graph.database.primitive_counts['NumberOfNodeIdsInUse'], 
    graph.database.primitive_counts['NumberOfRelationshipIdsInUse']))

In [ ]:
def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df
    
    
top_5 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science']
top_10 = ['Cell', 'Nature', 'Nature Biotechnology','Proceedings of the National Academy of Sciences of the United States of America','Science', 'Journal of the American Chemical Society', 'JAMA', 'The New England Journal of Medicine', 'Nature Genetics', 'Neuron']
top_42 = ['Angewandte Chemie','Blood','Cancer Cell','Cancer Discovery','Cancer Research','Cell','Cell Host & Microbe','Cell Metabolism','Cell Stem Cell','Chemistry & Biology','The EMBO Journal','Genes & Development','Immunity','Journal of Neurology','Journal of the American Chemical Society','JAMA','Journal of Biological Chemistry','Journal of Cell Biology','Journal of Clinical Investigation','Journal of Experimental Medicine','Journal of Medicinal Chemistry','The Lancet','Nature Cell Biology','Nature Chemical Biology','Nature Chemistry','Nature Medicine','Nature Methods','Nature','Nature Biotechnology','The New England Journal of Medicine','Neuron','Nature Genetics','Nature Immunology','Nature Neuroscience','Nature Structural & Molecular Biology','PLOS Biology','PLOS Genetics','PLOS Pathogens','Proceedings of the National Academy of Sciences of the United States of America','Science Signaling','Science Translational Medicine','Science']

In [ ]:
query = """
MATCH (q:Quanta)
WHERE q.venue in {}
RETURN 
    q.pageRank_2018 as pagerank, 
    q.articleRank2018 as articlerank, 
    q.num_profs as num_profs, 
    q.num_authors as num_authors
""".format(top_42)
df = run_query(query, graph, to_df=True)
df.dropna(inplace=True)
df.describe(include='all')

In [ ]:
df.info()

In [ ]:
g = sns.pairplot(np.log(df), markers='.', diag_kind='kde', height=3)

In [ ]:
impact_metrics = ['pagerank', 'articlerank']
collab_metrics = ['num_authors', 'num_profs']

for col in impact_metrics + collab_metrics:
    plt.figure()
    ax = sns.distplot(df[col], kde=False, bins=100)
    ax.set_yscale('log')

In [ ]:
sns.set_style('white')
g = sns.pairplot(df, y_vars=impact_metrics, x_vars=collab_metrics, 
             height=5, markers='.', plot_kws=dict(s=100))